In [ ]:
from custom.GeoSpatialEncoder import GeoSpatialEncoder
from custom.PC_Class import PC
import importlib
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
import os
datetime_cols = ['CREATIONDATETIME', 'LAAD_DATETIME_VAN', 'LAAD_DATETIME_TOT', 'LOS_DATETIME_VAN', 'LOS_DATETIME_TOT', '15CREATIONDATETIME']
direct = os.getcwd()
file_path = direct + "////data////vos_input_data////MultiHubData3_cleaned.csv" 
total_rows = sum(1 for row in open(file_path, 'r', encoding='utf-8'))
chunk_size = 10000  
tqdm.pandas(desc="Reading CSV")
chunks = pd.read_csv(file_path, chunksize=chunk_size, iterator=True, index_col = 0, parse_dates=datetime_cols)

df_orders = pd.concat(tqdm(chunks, total=total_rows//chunk_size))

# Convert the 'LOS_DATETIME_VAN' column to datetime format
for column in datetime_cols:
    print(f"column: {column}")
    df_orders[column] = pd.to_datetime(df_orders[column], errors='coerce')

print("Lenght of input data:", str(len(df_orders)))
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn")

In [59]:
def write_to_csv(df, file_name):
    path = # write path here
    
    file_name = path + file_name
    df.to_csv(file_name)

In [60]:
# list of all working days in 2023 in format YYYY-MM-DD strings
dates = pd.date_range(start='2023-01-01', end='2023-12-30', freq='B') # business days
datess = [str(date.date()) for date in dates]

In [ ]:
names = []
for date in datess:
        file_name = date + ".csv"
        # select only rows where delivery day is the date
        # Select only rows where delivery day is the specified date
        df_pre = df_orders[df_orders['LOS_DATETIME_VAN'].dt.date == pd.Timestamp(date).date()]
        df_pre_d = df_pre[
            (df_pre["AFHCODE"] == "d") & 
            (df_pre["LOS_CPC"] != "0") & 
            (df_pre["LAAD_CPC"] != "0") & 
            (df_pre["LOSRIT"] != "0") & 
            (df_pre["PALLETPLAATSEN"] < 32)
        ]
        columns_to_keep = [
            "SHIPMENTNUMBER", "STATUS", "CREATIONDATETIME", "AFHCODE", "OPDRACHTGEVERID", "OPDRACHTGEVERNAAM",
            "LAADZOEK", "LAADADRES", "LAADPLAATS", "LAADPC", "LAADLAND", "LAAD_DATETIME_VAN", "LAAD_DATETIME_TOT",
            "LAADPLANK", "LOSZOEK", "LOSADRES", "LOSPLAATS", "LOSPC", "LOSLAND", "LOS_DATETIME_VAN", "LOS_DATETIME_TOT",
            "LOSPLANK", "COLLIAANTAL", "COLLICODE", "PALLETPLAATSEN", "LAADRIT", "LOSRIT", "HAS_PICKUP_TRIP", 
            "HAS_DELIVERY_TRIP", "15CREATIONDATETIME", "LAAD_CPC", "LOS_CPC"
        ]
        group_by_columns = ['LAAD_CPC', 'OPDRACHTGEVERID', 'LOS_CPC']
        agg_dict = {col: 'last' for col in columns_to_keep if col not in group_by_columns and col != 'PALLETPLAATSEN' and col != 'SHIPMENTNUMBER'}
        agg_dict.update({
            'PALLETPLAATSEN': 'sum',
            'SHIPMENTNUMBER': lambda x: list(x),
            'COUNTER': 'count'  # Add a count for the number of orders
        })
        df_pre_d["COUNTER"] = df_pre_d["SHIPMENTNUMBER"]
        df_grouped = df_pre_d.groupby(group_by_columns).agg(agg_dict).reset_index()
        df_grouped.rename(columns={'COUNTER': 'ORDER_COUNT'}, inplace=True)
        df_grouped = df_grouped[df_grouped["PALLETPLAATSEN"] < 32]
        write_to_csv(df_grouped, file_name)
        names.append(file_name)
